# Class Generated Images

In [5]:
import os
import numpy as np
import torch
from torch.optim import SGD
from torchvision import models
import torch.nn as nn
from torch.autograd import Variable
import copy

import numpy as np
from PIL import Image
import matplotlib.cm as mpl_color_map
from matplotlib.colors import LogNorm
from astropy.io import fits
import matplotlib.pyplot as plt

from skimage.transform import resize
from scipy.ndimage import gaussian_filter
from CNN_Networks import OU200_CNN, VIS_CNN, JYH_CNN, OU66_CNN
from CNN_Layer_VIS import CNNLayerVisualization
from Generate_Class_Examples import ClassSpecificImageGeneration

## Load States
A dictionary containing all the weights for the CNNs used in the accompanning paper. 

In [6]:
loadStates = {'J': 'OU_J_Weights.pt',
              'Y': 'OU_Y_Weights.pt',
              'H': 'OU_H_Weights.pt',
              'JYH': 'OU_JYH_Weights.pt',
              'VIS': 'OU_VIS_Weights.pt',
              'OU-66': 'OU_66_Weights.pt',
              'OU-200': 'OU_200_Weights.pt'}

## Noise image maker
We create an image of the same dimensions as the inputs to our CNN containing random uniform noise. We save this random noise image to allow us to use the same input image for each class.

In [8]:
onebandnoise = np.random.random((66,66))
inputnoise = np.zeros((1,4,200,200))
for i in range(4):
    inputnoise[0,i,:,:] = resize(onebandnoise, (200,200))
np.save('input_noise_OU200', inputnoise)

## Making Class Generated Images
We first define a set of variables for this particluar run k (which is an identifying number for this run), lr (learning rate), range_Images (how many interations this process runs for), and rand_loc (location of the intial input image). 

The CNN is selected and loaded with the trained weights. The random image is loaded. The file locations for the output of this notebook are specificied depending on the target class. These input variables are given to the ClassSpecificImageGeneration() method and the noise image is updated to activate the target class. This function saves the numpy file of the image and a single band image every 10 interations. This process is carried out for both target classes.

In [13]:
k = 0
lr = 0.1
range_Images = 100
rand_loc = 'input_noise_OU200.npy'
for j in range(2):
                target_class = j 
                model = OU200_CNN()
                criterion = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
                model.load_state_dict(torch.load(loadStates['OU-200'],map_location=torch.device('cpu')))

                input_img = np.load(rand_loc)


                if target_class == 0:
                        file_loc = 'Non_Lens_Gifs/non_lenses'
                else:
                        file_loc = 'Lens_Gifs/Lenses'
 
                loc = 'generated/OU200/'+str(range_Images)+'/'+file_loc+'_'+str(lr)+'_No_'+str(k)+'C_SameRandom4AllBands/'
                if not os.path.exists(loc):
                    os.makedirs(loc)
                  
                pretrained_model = model
                csig = ClassSpecificImageGeneration(pretrained_model, target_class, lr, loc, input_img, range_Images)
                csig.generate()

Iteration: 1 Loss 0.25 Acc: [0.37785983 0.62214017]
Iteration: 2 Loss 0.17 Acc: [0.38496557 0.61503446]
Iteration: 3 Loss -0.34 Acc: [0.6064154 0.3935846]
Iteration: 4 Loss -0.50 Acc: [0.68301684 0.3169831 ]
Iteration: 5 Loss -0.64 Acc: [0.7460161  0.25398383]
Iteration: 6 Loss -0.67 Acc: [0.7649195 0.2350805]
Iteration: 7 Loss -0.76 Acc: [0.7913174  0.20868258]
Iteration: 8 Loss -0.91 Acc: [0.82867074 0.17132926]
Iteration: 9 Loss -0.96 Acc: [0.8441642  0.15583578]
Iteration: 10 Loss -1.05 Acc: [0.857437   0.14256305]
Iteration: 11 Loss -1.02 Acc: [0.8723683  0.12763171]
Iteration: 12 Loss -1.33 Acc: [0.91540956 0.08459049]
Iteration: 13 Loss -1.30 Acc: [0.91500586 0.08499417]
Iteration: 14 Loss -1.40 Acc: [0.9294658  0.07053424]
Iteration: 15 Loss -1.46 Acc: [0.9378548 0.0621452]
Iteration: 16 Loss -1.60 Acc: [0.9521125  0.04788755]
Iteration: 17 Loss -1.60 Acc: [0.946157   0.05384297]
Iteration: 18 Loss -1.61 Acc: [0.9556216  0.04437842]
Iteration: 19 Loss -1.80 Acc: [0.96558315 0.0

<Figure size 432x288 with 0 Axes>

## Generating Class Activating Images for each Conv2D layer 

This works on the same principle as Class Generated Images. However instead of focusing on the output layer, we focus on the Conv2D layers and all the filters within these layers. These layers are at different locations for the different architectures used within this work. 

This process creates images that are the same dimensions as the inputs to OU-200 that highly activate a specific layer and filter. This can indicate what each filter in the Conv2D layers is detecting.

In [ ]:
model = OU66_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.load_state_dict(torch.load(loadStates['OU-66'],map_location=torch.device('cpu')))

In [ ]:
for i in [0,4,7,10,13,17,20,23,26]:
    for j in range(model.layer[i].weight.data.shape[0]):
        cnn_layer = i
        filter_pos = j
        lr = 0.1
        steps = 50
    # Fully connected layer is not needed
        pretrained_model = model
        input_Size = [4,200,200]
        path = 'generated/OU66_Filters_Run1'
        if not os.path.exists(path):
                    os.makedirs(path)
    #pretrained_model = models.vgg16(pretrained=True).features
    
        layer_vis = CNNLayerVisualization(pretrained_model, cnn_layer, filter_pos, lr, steps,path,input_Size)

    # Layer visualization with pytorch hooks
        layer_vis.visualise_layer_with_hooks()

In [ ]:
model = OU200_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.load_state_dict(torch.load(loadStates['OU-200'],map_location=torch.device('cpu')))

In [ ]:
for i in [0,4,8,12,16,19]:
    for j in range(model.layer[i].weight.data.shape[0]):
        cnn_layer = i
        filter_pos = j
        lr = 0.1
        steps = 50
    # Fully connected layer is not needed
        pretrained_model = model
        input_Size = [4,200,200]
        path = 'generated/OU200_Filters_Run1'
        if not os.path.exists(path):
                    os.makedirs(path)
    #pretrained_model = models.vgg16(pretrained=True).features
        layer_vis = CNNLayerVisualization(pretrained_model, cnn_layer, filter_pos, lr, steps,path,input_Size)

    # Layer visualization with pytorch hooks
        layer_vis.visualise_layer_with_hooks()